<a href="https://colab.research.google.com/github/sladem-tox/Tox_data/blob/main/AtomPairSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Load data from Github
import pandas as pd
df = pd.read_csv("https://github.com/sladem-tox/Tox_data/raw/main/Organophosphate_ActivityValues.csv")

See:
https://greglandrum.github.io/rdkit-blog/posts/2023-01-18-fingerprint-generator-tutorial.html


In [3]:
!pip install -q datamol
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [8]:
import rdkit
print(rdkit.__version__)
%pylab inline

2024.09.6
Populating the interactive namespace from numpy and matplotlib


In [11]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
from rdkit.Chem import rdFingerprintGenerator
from rdkit.DataStructs.cDataStructs import TanimotoSimilarity

# Create AtomPair fingerprint generator
apgen = rdFingerprintGenerator.GetAtomPairGenerator(fpSize=2048)

# Convert SMILES to RDKit molecules
df['Mol'] = df['SMILES'].apply(Chem.MolFromSmiles)

# Generate AtomPair fingerprints
df['AP_FP'] = df['Mol'].apply(lambda mol: apgen.GetFingerprint(mol))

# Compare all fingerprints: compute pairwise Tanimoto similarity
n = len(df)
similarity_matrix = pd.DataFrame(index=df.index, columns=df.index)

for i in range(n):
    fp_i = df['AP_FP'].iloc[i]
    for j in range(i, n):
        fp_j = df['AP_FP'].iloc[j]
        sim = TanimotoSimilarity(fp_i, fp_j)
        similarity_matrix.iat[i, j] = sim
        similarity_matrix.iat[j, i] = sim  # symmetric

# Ensure numeric type
similarity_matrix = similarity_matrix.astype(float)

# Print result
print(similarity_matrix)


          0         1         2         3         4         5         6    \
0    1.000000  0.730769  0.623932  0.541985  0.496503  0.473684  0.672566   
1    0.730769  1.000000  0.775862  0.664122  0.586207  0.558442  0.694215   
2    0.623932  0.775862  1.000000  0.781250  0.678322  0.612903  0.557971   
3    0.541985  0.664122  0.781250  1.000000  0.800000  0.712418  0.483660   
4    0.496503  0.586207  0.678322  0.800000  1.000000  0.826667  0.422619   
..        ...       ...       ...       ...       ...       ...       ...   
273  0.017857  0.056497  0.059140  0.111702  0.105000  0.126829  0.048128   
274  0.017442  0.038043  0.080645  0.115183  0.108374  0.124402  0.030928   
275  0.017647  0.038462  0.064171  0.122340  0.109453  0.125604  0.031250   
276  0.016043  0.035176  0.058824  0.106796  0.137441  0.136364  0.028708   
277  0.020000  0.042654  0.059908  0.105023  0.118943  0.157895  0.031532   

          7         8         9    ...       268       269       270  \
0  